In [1]:
import pandas as pd


def data_loader(path):
    table = pd.read_csv(path)
    return table


In [2]:
table = data_loader('../archive (1)/BrentOilPrices.csv')

In [3]:
from sklearn.preprocessing import MinMaxScaler

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, LSTM


def index_change(table, col_name, col_target_name):
    table = table.set_index(col_name)
    table.index = pd.to_datetime(table.index)
    for col in table.columns:
        scaler = MinMaxScaler()
        table[col] = scaler.fit_transform(table)
    x = table[col_target_name].values.tolist()
    return x

In [5]:
x = index_change(table, 'Date', 'Price')

In [8]:
def train_test(data, data_cut, step):
    train, test = data[:data_cut], data[data_cut:]
    train = np.append(train, np.repeat(train[-1], step))
    test = np.append(test, np.repeat(test[-1], step))
    return train, test

In [10]:
train, test = train_test(x, 650, 4)

In [11]:
def convert_to_matrix(data, step):
    x, y = [], []
    for i in range(len(data) - step):
        d = i + step
        x.append(data[i:d])
        y.append(data[d])
    return np.array(x), np.array(y)


In [12]:
train_x, train_y = convert_to_matrix(train, 4)

In [13]:
test_x, test_y = convert_to_matrix(test, 4)

In [14]:
def reshape(train_x, test_x):
    train_x = np.reshape(train_x, (train_x.shape[0], 1, train_x.shape[1]))
    test_x = np.reshape(test_x, (test_x.shape[0], 1, test_x.shape[1]))
    return train_x, test_x

In [15]:
train_x, test_x = reshape(train_x,test_x)

In [16]:
def create_model():
    model = Sequential()
    model.add(LSTM(64, activation='relu', input_shape=(1, 4)))
    model.add(Dense(36, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['acc'])
    return model

In [17]:
model = create_model()

In [18]:
def model_fit(train_x, train_y, epochs_count):
    model.fit(train_x, train_y, epochs=epochs_count)

In [19]:
model_fit(train_x, train_y, 10)

Epoch 1/10
21/21 [==============================] - 1s 1ms/step - loss: 0.0015 - acc: 0.0000e+00
Epoch 2/10
21/21 [==============================] - 0s 1ms/step - loss: 2.4599e-04 - acc: 0.0000e+00
Epoch 3/10
21/21 [==============================] - 0s 1ms/step - loss: 2.0623e-04 - acc: 0.0000e+00
Epoch 4/10
21/21 [==============================] - 0s 1ms/step - loss: 2.0110e-04 - acc: 0.0000e+00
Epoch 5/10
21/21 [==============================] - 0s 1ms/step - loss: 1.9923e-04 - acc: 0.0000e+00
Epoch 6/10
21/21 [==============================] - 0s 1ms/step - loss: 1.8831e-04 - acc: 0.0000e+00
Epoch 7/10
21/21 [==============================] - 0s 1ms/step - loss: 1.7759e-04 - acc: 0.0000e+00
Epoch 8/10
21/21 [==============================] - 0s 1ms/step - loss: 1.6510e-04 - acc: 0.0000e+00
Epoch 9/10
21/21 [==============================] - 0s 1ms/step - loss: 1.4790e-04 - acc: 0.0000e+00
Epoch 10/10
21/21 [==============================] - 0s 988us/step - loss: 1.2752e-04 - acc: 0.